In [1]:
# ALexNet 是几个卷积池化堆叠后连接几个全连接层
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets import CIFAR10


In [19]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 第一层是5*5 的卷积，输入的是channels 是3，输出的channels是64，步长为1，没有padding
        self.conv1=nn.Sequential(
        nn.Conv2d(3,64,5),
        nn.ReLU(True))
        # 第二层是3*3 的池化，步长为2，没有padding
        self.max_pool1=nn.MaxPool2d(3,2)
        # 第三层是5*5 的卷积，输入的channels 是64，输出的channels 是64，步长是1，没有padding。
        
        self.conv2=nn.Sequential(
            nn.Conv2d(64,64,5,1),
            nn.ReLU(True)
        
        )
        # 第四层 是3*3 的池化，步长为2，没有padding
        self.max_pool2=nn.MaxPool2d(3,2)
        
        # 第五层是全连接层，输入是2，输出是384
        self.fc1=nn.Sequential(
            nn.Linear(1024,384),
            nn.ReLU(True)
            
            
        )
        # 第六层全连接层，输入384，输出192
        self.fc2=nn.Sequential(
            nn.Linear(384,192),
            nn.ReLU(True)
        
        )
        # 第七层全连接层，输入192，输出是10
        self.fc3=nn.Linear(192,10)
    def forward(self,x):
        x=self.conv1(x)
        x=self.max_pool1(x)
        x=self.conv2(x)
        x=self.max_pool2(x)
        # 将矩阵拉平
        x=x.view(x.shape[0],-1)
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        return x


In [20]:
alexnet=AlexNet()

In [21]:
alexnet

AlexNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (max_pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (max_pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=1024, out_features=384, bias=True)
    (1): ReLU(inplace)
  )
  (fc2): Sequential(
    (0): Linear(in_features=384, out_features=192, bias=True)
    (1): ReLU(inplace)
  )
  (fc3): Linear(in_features=192, out_features=10, bias=True)
)

In [22]:
# 输入一张32*32 的图片，看看输出
input_demo=Variable(torch.zeros(1,3,32,32))
output_demo=alexnet(input_demo)
print(output_demo.shape)

torch.Size([1, 10])


In [ ]:
from utils import train

def data_tf(x):
    x=np.array(x,dtype="float32")/255
    x=(x-0.5)/0.5
    x=x.transpose((2,0,1) ) # 将channel 放到第一维，
    x=torch.from_numpy(x)
    return x

train_set=CIFAR10("./data",train=True,transform=data_tf)
train_data=torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)
test_set=CIFAR10("./data",train=False,transform=data_tf)
test_data=torch.utils.data.DataLoader(test_set,batch_size=128,shuffle=False)
net=AlexNet()
optimizer=torch.optim.SGD(net.parameters(),lr=1e-1)
criterion=nn.CrossEntropyLoss()


In [ ]:
train(net,train_data,test_data,20,optimizer,criterion)